# 🔧 CELL 0 — Thiết lập biến môi trường

## Mục đích
Cell này thiết lập các biến môi trường **TRƯỚC KHI** import các thư viện để tránh các lỗi xung đột giữa PyTorch và TensorFlow.

## Giải thích chi tiết các biến:

| Biến môi trường | Giá trị | Mục đích |
|-----------------|---------|----------|
| `TRANSFORMERS_NO_TF` | "1" | Chặn Transformers tự động import TensorFlow |
| `TRANSFORMERS_NO_FLAX` | "1" | Chặn Transformers tự động import Flax (JAX framework) |
| `TF_CPP_MIN_LOG_LEVEL` | "3" | Tắt các log warning của TensorFlow C++ backend |
| `TOKENIZERS_PARALLELISM` | "false" | Tắt tokenization song song để tránh deadlock trong multi-processing |

## Tại sao cần thiết?
- **Tránh lỗi protobuf/MessageFactory**: Khi TensorFlow và PyTorch cùng tồn tại, có thể xảy ra xung đột protobuf version
- **Giảm warning không cần thiết**: Các log của TF có thể gây nhiễu output
- **Ổn định môi trường**: Đảm bảo chỉ sử dụng PyTorch backend cho toàn bộ pipeline

## Lưu ý
- **Phải chạy cell này TRƯỚC** khi import bất kỳ thư viện nào khác
- Nếu bỏ qua cell này, có thể gặp lỗi MessageFactory hoặc protobuf conflicts

In [2]:
import sys, torch
import numpy as np

import transformers, datasets, accelerate

print("Python:", sys.version.split()[0])
print("Torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("Accelerate:", accelerate.__version__)

Python: 3.11.13
Torch: 2.6.0+cu124
CUDA: True
GPU: Tesla T4
Transformers: 4.53.3
Datasets: 4.4.1
Accelerate: 1.9.0


# 📊 CELL 1 — Kiểm tra môi trường và GPU

## Mục đích
Kiểm tra và hiển thị thông tin về môi trường runtime để đảm bảo:
- Phiên bản Python và các thư viện tương thích
- GPU có sẵn để tăng tốc training
- Tránh lỗi compatibility giữa các version

## Các thông tin được kiểm tra:

| Thông tin | Ý nghĩa |
|-----------|---------|
| `Python: 3.11.13` | Phiên bản Python đang chạy |
| `Torch: 2.6.0+cu124` | PyTorch 2.6.0 với CUDA 12.4 |
| `CUDA: True` | GPU có sẵn và được kích hoạt |
| `GPU: Tesla T4` | Card đồ họa (16GB VRAM trên Kaggle) |
| `Transformers: 4.53.3` | Hugging Face Transformers library |
| `Datasets: 4.4.1` | Hugging Face Datasets library |
| `Accelerate: 1.9.0` | Library hỗ trợ distributed training |

## Lưu ý quan trọng
- **Tesla T4** là GPU phổ biến trên Kaggle/Colab với 16GB VRAM
- Nếu `CUDA: False`, model sẽ chạy trên CPU (chậm hơn 10-100x)
- Transformers 4.53.3 là version mới nhất, tương thích với mBART-50

# 📦 CELL 2 — Cài đặt packages bổ sung

## Mục đích
Cài đặt các package cần thiết nếu chưa có trong môi trường Kaggle.

## Packages được cài:

| Package | Mục đích | Bắt buộc? |
|---------|----------|-----------|
| `sacrebleu` | Tính điểm BLEU chuẩn cho đánh giá dịch máy | ✅ Bắt buộc |
| `hf_transfer` | Tăng tốc download/upload model từ Hugging Face Hub | ⚪ Tùy chọn |

## Cách hoạt động:
1. `importlib.util.find_spec()` kiểm tra package đã được cài chưa
2. Nếu chưa có → gọi `pip install -q` (quiet mode)
3. Nếu đã có → bỏ qua, in thông báo "already installed"

## Về sacrebleu
- **SacreBLEU** là công cụ chuẩn để đánh giá chất lượng dịch máy
- Tính điểm BLEU (Bilingual Evaluation Understudy) từ 0-100
- Điểm càng cao = bản dịch càng giống với reference
- Cung cấp kết quả consistent và reproducible (không phụ thuộc vào tokenization)

In [3]:
import importlib.util, subprocess

def pip_install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# sacrebleu: bắt buộc
if importlib.util.find_spec("sacrebleu") is None:
    print("[INFO] Installing sacrebleu ...")
    pip_install("sacrebleu")
else:
    print("[OK] sacrebleu already installed")

# hf_transfer: optional tăng tốc push/pull
if importlib.util.find_spec("hf_transfer") is None:
    print("[INFO] Installing hf_transfer (optional) ...")
    pip_install("hf_transfer")
else:
    print("[OK] hf_transfer already installed")

[INFO] Installing sacrebleu ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00
[OK] hf_transfer already installed


In [4]:
import re, math, random, hashlib
from pathlib import Path
from typing import List, Dict, Any

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
import sacrebleu

print("[OK] Imports done.")

E0000 00:00:1765904980.968127      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765904981.036662      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[OK] Imports done.


In [5]:
from pathlib import Path

def find_dataset_dir() -> Path:
    base = Path("/kaggle/input")
    for d in base.glob("*"):
        if d.is_dir() and (d / "train.en.txt").exists() and (d / "train.vi.txt").exists():
            return d
    for d in base.glob("*/*"):
        if d.is_dir() and (d / "train.en.txt").exists() and (d / "train.vi.txt").exists():
            return d
    raise FileNotFoundError("Could not find dataset folder containing train.en.txt/train.vi.txt")

DATA_DIR = find_dataset_dir()
print("[OK] Found dataset dir:", DATA_DIR)

TRAIN_EN = DATA_DIR / "train.en.txt"
TRAIN_VI = DATA_DIR / "train.vi.txt"
TEST_EN  = DATA_DIR / "public_test.en.txt"
TEST_VI  = DATA_DIR / "public_test.vi.txt"

for p in [TRAIN_EN, TRAIN_VI, TEST_EN, TEST_VI]:
    print("[CHECK]", p.name, "->", "OK" if p.exists() else "NOT FOUND")

[OK] Found dataset dir: /kaggle/input/databaitoanphu
[CHECK] train.en.txt -> OK
[CHECK] train.vi.txt -> OK
[CHECK] public_test.en.txt -> OK
[CHECK] public_test.vi.txt -> OK


In [6]:
def read_lines(path: Path, encoding="utf-8"):
    with open(path, "r", encoding=encoding, errors="replace") as f:
        return [line.rstrip("\n") for line in f]

def load_parallel(en_path: Path, vi_path: Path, name="data"):
    en_lines = read_lines(en_path)
    vi_lines = read_lines(vi_path)
    n = min(len(en_lines), len(vi_lines))
    if len(en_lines) != len(vi_lines):
        print(f"[WARN] {name} mismatch -> EN={len(en_lines):,}, VI={len(vi_lines):,}. Using first {n:,} pairs.")
    else:
        print(f"[INFO] {name} lines -> {n:,} pairs.")
    return list(zip(en_lines[:n], vi_lines[:n]))

train_pairs_raw = load_parallel(TRAIN_EN, TRAIN_VI, "train_raw")
test_pairs_raw  = load_parallel(TEST_EN,  TEST_VI,  "test_raw")

print("[SAMPLE train_raw]", train_pairs_raw[0])
print("[SAMPLE test_raw ]", test_pairs_raw[0])

[INFO] train_raw lines -> 500,000 pairs.
[INFO] test_raw lines -> 3,000 pairs.
[SAMPLE train_raw] ('To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital', 'Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên')
[SAMPLE test_raw ] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')


In [7]:
import re, hashlib

def basic_clean(s: str) -> str:
    s = "" if s is None else s
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

MIN_CHARS = 2
MAX_CHARS = 400

def is_good_pair(src: str, tgt: str) -> bool:
    if not src or not tgt:
        return False
    if len(src) < MIN_CHARS or len(tgt) < MIN_CHARS:
        return False
    if len(src) > MAX_CHARS or len(tgt) > MAX_CHARS:
        return False
    return True

def clean_filter_dedup(pairs, name="data"):
    # clean + filter
    cleaned = []
    for src, tgt in pairs:
        src = basic_clean(src)
        tgt = basic_clean(tgt)
        if is_good_pair(src, tgt):
            cleaned.append((src, tgt))
    print(f"[INFO] {name}: after clean+filter -> {len(cleaned):,} pairs")

    # dedup
    seen = set()
    dedup = []
    for src, tgt in cleaned:
        h = hashlib.md5((src + "\t" + tgt).encode("utf-8")).hexdigest()
        if h not in seen:
            seen.add(h)
            dedup.append((src, tgt))
    print(f"[INFO] {name}: after dedup       -> {len(dedup):,} pairs")
    return dedup

train_pairs = clean_filter_dedup(train_pairs_raw, "train")
test_pairs  = clean_filter_dedup(test_pairs_raw,  "test")

print("[SAMPLE clean train]", train_pairs[0])
print("[SAMPLE clean test ]", test_pairs[0])

[INFO] train: after clean+filter -> 489,008 pairs
[INFO] train: after dedup       -> 340,522 pairs
[INFO] test: after clean+filter -> 2,946 pairs
[INFO] test: after dedup       -> 2,943 pairs
[SAMPLE clean train] ('To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital', 'Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên')
[SAMPLE clean test ] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')


In [8]:
import random
from datasets import Dataset, DatasetDict

SEED = 42
N_TOTAL = 10000

TRAIN_RATIO = 0.9  # 90/10

# 1) shuffle train_pairs
rng = random.Random(SEED)
rng.shuffle(train_pairs)

# 2) lấy 5000 (nếu không đủ thì lấy hết)
n_take = min(N_TOTAL, len(train_pairs))
subset = train_pairs[:n_take]
print(f"[INFO] Train pairs available: {len(train_pairs):,}")
print(f"[INFO] Taking from train     : {n_take:,} pairs")

# 3) split 90/10
n_train = int(n_take * TRAIN_RATIO)
n_val = n_take - n_train

train_subset = subset[:n_train]
val_subset   = subset[n_train:]

print(f"[INFO] Split -> train={len(train_subset):,} | val={len(val_subset):,}")

# 4) build datasets
train_ds = Dataset.from_dict({
    "src_text": [s for s, _ in train_subset],
    "tgt_text": [t for _, t in train_subset],
})
val_ds = Dataset.from_dict({
    "src_text": [s for s, _ in val_subset],
    "tgt_text": [t for _, t in val_subset],
})

# test giữ nguyên từ public_test
test_ds = Dataset.from_dict({
    "src_text": [s for s, _ in test_pairs],
    "tgt_text": [t for _, t in test_pairs],
})

ds = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})
print(ds)

print("[SAMPLE TRAIN]", ds["train"][0])
print("[SAMPLE VAL  ]", ds["validation"][0])
print("[SAMPLE TEST ]", ds["test"][0])

[INFO] Train pairs available: 340,522
[INFO] Taking from train     : 10,000 pairs
[INFO] Split -> train=9,000 | val=1,000
DatasetDict({
    train: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 9000
    })
    validation: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['src_text', 'tgt_text'],
        num_rows: 2943
    })
})
[SAMPLE TRAIN] {'src_text': 'Monitor and control ICP using sedatives, endotracheal intubation, hyperventilation, hydration, diuretics, measures to control blood pressure, and sometimes corticosteroids.', 'tgt_text': 'Theo dõi và kiểm soát ICP bằng cách sử dụng thuốc an thần, đặt nội khí quản, tăng thông khí, hydrat hoá, thuốc lợi tiểu, các biện pháp để kiểm soát huyết áp, và đôi khi là corticosteroid.'}
[SAMPLE VAL  ] {'src_text': 'The most obvious result of nonadherence is that the disorder may not be relieved or cured.', 'tgt_text': 'Kết quả rõ ràng nhất của sự kh

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

MODEL_ID = "facebook/mbart-large-50-many-to-many-mmt"
SRC_LANG = "en_XX"
TGT_LANG = "vi_VN"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

# set src/tgt language (QUAN TRỌNG)
tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG   # <-- FIX KeyError: None

forced_bos_token_id = tokenizer.lang_code_to_id[TGT_LANG]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("[OK] tokenizer/model loaded")
print("[INFO] tokenizer.src_lang:", tokenizer.src_lang)
print("[INFO] tokenizer.tgt_lang:", tokenizer.tgt_lang)
print("[INFO] forced_bos_token_id:", forced_bos_token_id)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

[OK] tokenizer/model loaded
[INFO] tokenizer.src_lang: en_XX
[INFO] tokenizer.tgt_lang: vi_VN
[INFO] forced_bos_token_id: 250024


In [10]:
# đảm bảo ds đã có ở cell split 5000 (train/validation/test)
assert "ds" in globals(), "You must run the data-loading/splitting cell to create `ds` first!"
assert "tokenizer" in globals(), "You must run the tokenizer/model loading cell first!"

MAX_SRC_LEN = 256
MAX_TGT_LEN = 256

def preprocess_batch(batch):
    tokenizer.src_lang = SRC_LANG

    model_inputs = tokenizer(
        batch["src_text"],
        max_length=MAX_SRC_LEN,
        truncation=True,
        padding=False,
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt_text"],
            max_length=MAX_TGT_LEN,
            truncation=True,
            padding=False,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tok = ds["train"].map(preprocess_batch, batched=True, remove_columns=ds["train"].column_names)
val_tok   = ds["validation"].map(preprocess_batch, batched=True, remove_columns=ds["validation"].column_names)
test_tok  = ds["test"].map(preprocess_batch, batched=True, remove_columns=ds["test"].column_names)

print("[OK] Tokenized sizes:",
      "train", len(train_tok),
      "| val", len(val_tok),
      "| test", len(test_tok))

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2943 [00:00<?, ? examples/s]

[OK] Tokenized sizes: train 9000 | val 1000 | test 2943


In [11]:
@torch.inference_mode()
def translate_en2vi(texts, num_beams=5, length_penalty=1.0, no_repeat_ngram_size=3, max_new_tokens=128):
    model.eval()
    tokenizer.src_lang = SRC_LANG
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SRC_LEN).to(model.device)
    out = model.generate(
        **enc,
        forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG],
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        max_new_tokens=max_new_tokens,
        early_stopping=True
    )
    return tokenizer.batch_decode(out, skip_special_tokens=True)

def bleu_on_test(max_samples=None):
    srcs = ds["test"]["src_text"]
    refs = ds["test"]["tgt_text"]

    if max_samples is not None:
        srcs = srcs[:max_samples]
        refs = refs[:max_samples]

    hyps = []
    bs = 16 if torch.cuda.is_available() else 4
    for i in range(0, len(srcs), bs):
        hyps.extend(translate_en2vi(srcs[i:i+bs]))

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    print(f"[TEST] sacreBLEU ({len(srcs)} samples): {bleu:.2f}")
    return bleu

_ = bleu_on_test(max_samples=200)   # đổi None để chạy full public_test

[TEST] sacreBLEU (200 samples): 26.20


# 🎯 CELL 13 — Training với Early Stopping và Best Model Selection

## Mục đích
Train model mBART-50 trên 10,000 mẫu với cấu hình tối ưu cho thí nghiệm 2.

## Điểm khác biệt so với Thí nghiệm 1:

| Tham số | Thí nghiệm 1 (file 1) | Thí nghiệm 2 (file này) |
|---------|------------------------|-------------------------|
| Dữ liệu training | 20,000 mẫu | 10,000 mẫu |
| Train/Val split | 18K/2K | 9K/1K |
| Learning rate | 3e-5 | 1e-5 (thấp hơn 3x) |
| Số epochs | 3 | 7 |
| Early stopping | Không | Có (patience=2) |
| Save strategy | No checkpoint | Save best by eval_loss |
| BLEU cuối cùng | 41.51 | 38.09 |

## Chiến lược training:

### 1. Lower Learning Rate (1e-5)
- LR thấp hơn để training ổn định hơn với ít data hơn
- Tránh overfitting khi chỉ có 10K mẫu

### 2. Early Stopping
```python
EarlyStoppingCallback(early_stopping_patience=2)
```
- Dừng training nếu validation loss không cải thiện sau 2 epochs
- Tiết kiệm thời gian và tránh overfitting

### 3. Best Model Selection
```python
load_best_model_at_end=True
metric_for_best_model="eval_loss"
greater_is_better=False
save_total_limit=1  # chỉ giữ checkpoint tốt nhất
```
- Model cuối cùng là checkpoint có val_loss thấp nhất
- Không phải model ở epoch cuối

## Kết quả Training:

### Loss theo epochs:
| Epoch | Train Loss | Val Loss | Cải thiện |
|-------|------------|----------|-----------|
| 1 | 1.9404 | 1.7236 | - |
| 2 | 1.5553 | 1.6193 | 6.1% |
| 3 | 1.3820 | 1.5737 | 2.8% |
| 4 | 1.2750 | 1.5550 | 1.2% |
| **5** | **1.1969** | **1.5457** | **0.6%** ← Best |
| 6 | 1.1411 | 1.5458 | 0.0% |
| 7 | 1.1068 | 1.5457 | 0.0% |

### Quan sát:
- **Best epoch**: Epoch 5 với val_loss = 1.5457
- **Plateau**: Val loss không cải thiện từ epoch 5→7
- **Training time**: ~3.2 giờ trên Tesla T4
- **Final BLEU**: 38.09 (thấp hơn Thí nghiệm 1 do ít data hơn)

## Files được lưu:
1. `mbart50_envi/` - Model weights (best checkpoint)
2. `train_val_loss.tsv` - Training history
3. Pushed lên HuggingFace: `ngothuyet/mbart50-envi-e7`

In [12]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()

# tên secret của anh là HF_TOKEN (đúng như panel)
hf_token = user_secrets.get_secret("HF_TOKEN")

login(token=hf_token)
print("[OK] Logged in to Hugging Face.")

[OK] Logged in to Hugging Face.


In [13]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# HF_REPO = "ngothuyet/mbart50-envi-e7"

# tokenizer = AutoTokenizer.from_pretrained(HF_REPO, use_fast=False)
# model = AutoModelForSeq2SeqLM.from_pretrained(HF_REPO)

# tokenizer.src_lang = "en_XX"
# tokenizer.tgt_lang = "vi_VN"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# print("[OK] Loaded model from HF:", HF_REPO)

In [14]:
import os, inspect
import torch
import pandas as pd
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
)

# ====== CHECK ======
assert "model" in globals(), "Missing `model`"
assert "tokenizer" in globals(), "Missing `tokenizer`"
assert "train_tok" in globals(), "Missing `train_tok`"
assert "val_tok" in globals(), "Missing `val_tok`"
assert "SRC_LANG" in globals() and "TGT_LANG" in globals(), "Missing SRC_LANG/TGT_LANG"

tokenizer.src_lang = SRC_LANG
tokenizer.tgt_lang = TGT_LANG

OUTPUT_DIR  = "mbart50_envi"
NUM_EPOCHS  = 7

TRAIN_BS = 2
EVAL_BS  = 2
GRAD_ACC = 8
LR       = 1e-5

HF_REPO_ID = "ngothuyet/mbart50-envi-e7"
HF_PRIVATE = True

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

ta_kwargs = dict(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,

    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    warmup_ratio=0.03,
    weight_decay=0.01,

    predict_with_generate=False,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
    
    # ✅ SAVE BEST theo VAL LOSS (eval_loss)
    save_strategy="epoch",        # cần save để chọn best
    save_only_model=True,         # tránh lỗi optimizer.pt
    save_total_limit=1,           # chỉ giữ 1 cái tốt nhất

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

sig = inspect.signature(Seq2SeqTrainingArguments.__init__).parameters

if "logging_strategy" in sig:
    ta_kwargs["logging_strategy"] = "epoch"

if "evaluation_strategy" in sig:
    ta_kwargs["evaluation_strategy"] = "epoch"
elif "eval_strategy" in sig:
    ta_kwargs["eval_strategy"] = "epoch"
else:
    raise ValueError("Transformers version: missing evaluation_strategy/eval_strategy")

ta_kwargs.update(dict(
    push_to_hub=True,
    hub_model_id=HF_REPO_ID,
    hub_private_repo=HF_PRIVATE,
    hub_strategy="end",           # chỉ push cuối
))

training_args = Seq2SeqTrainingArguments(**ta_kwargs)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)], 
)

print("[INFO] Start training...")
train_result = trainer.train()
print("\n===== TRAIN DONE =====")
print(train_result)

print("\n===== BEST (by eval_loss) =====")
print("best_model_checkpoint:", trainer.state.best_model_checkpoint)
print("best_metric(eval_loss):", trainer.state.best_metric)

print("\n===== FINAL EVAL (VAL) on BEST =====")
final_metrics = trainer.evaluate()
for k, v in final_metrics.items():
    print(f"{k}: {v}")

# Lưu BEST local (trainer.model đã là best vì load_best_model_at_end=True)
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("[OK] Saved BEST model/tokenizer to:", OUTPUT_DIR)

# TSV train/val loss như cũ
logs = trainer.state.log_history
train_loss_by_epoch = {}
val_loss_by_epoch = {}

for row in logs:
    if "epoch" in row and "loss" in row and "eval_loss" not in row:
        train_loss_by_epoch[row["epoch"]] = row["loss"]
    if "epoch" in row and "eval_loss" in row:
        val_loss_by_epoch[row["epoch"]] = row["eval_loss"]

epochs = sorted(set(list(train_loss_by_epoch.keys()) + list(val_loss_by_epoch.keys())))
df = pd.DataFrame({
    "epoch": epochs,
    "train_loss": [train_loss_by_epoch.get(e, None) for e in epochs],
    "val_loss": [val_loss_by_epoch.get(e, None) for e in epochs],
})

tsv_path = os.path.join(OUTPUT_DIR, "train_val_loss.tsv")
df.to_csv(tsv_path, sep="\t", index=False)
print("[OK] Saved TSV:", tsv_path)
display(df)

# ✅ Push BEST + TSV (không còn push “bản cuối” theo nghĩa last-epoch)
trainer.push_to_hub(commit_message=f"Push BEST by eval_loss={trainer.state.best_metric} + train_val_loss.tsv")
print("[OK] Pushed BEST model + TSV to Hugging Face.")


[INFO] Start training...


/tmp/ipykernel_47/1399803177.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.940400,1.723647
2,1.555300,1.619265
3,1.382000,1.573678
4,1.275000,1.555019
5,1.196900,1.545686
6,1.141100,1.545778
7,1.106800,1.547994


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



===== TRAIN DONE =====
TrainOutput(global_step=1974, training_loss=1.3710759867288422, metrics={'train_runtime': 11349.5282, 'train_samples_per_second': 5.551, 'train_steps_per_second': 0.174, 'total_flos': 7306723507765248.0, 'train_loss': 1.3710759867288422, 'epoch': 7.0})

===== BEST (by eval_loss) =====
best_model_checkpoint: mbart50_envi/checkpoint-1410
best_metric(eval_loss): 1.5456864833831787

===== FINAL EVAL (VAL) on BEST =====


...

[OK] Saved BEST model/tokenizer to: mbart50_envi
[OK] Saved TSV: mbart50_envi/train_val_loss.tsv


,epoch,train_loss,val_loss
0,1.0,1.9404,1.723647
1,2.0,1.5553,1.619265
2,3.0,1.3820,1.573678
3,4.0,1.2750,1.555019
4,5.0,1.1969,1.545686
5,6.0,1.1411,1.545778
6,7.0,1.1068,1.545686


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[OK] Pushed BEST model + TSV to Hugging Face.


In [15]:
# lấy log_history và gom theo epoch
logs = trainer.state.log_history

train_loss_by_epoch = {}
val_loss_by_epoch = {}

for row in logs:
    if "epoch" in row and "loss" in row and "eval_loss" not in row:
        train_loss_by_epoch[row["epoch"]] = row["loss"]
    if "epoch" in row and "eval_loss" in row:
        val_loss_by_epoch[row["epoch"]] = row["eval_loss"]

epochs = sorted(set(list(train_loss_by_epoch.keys()) + list(val_loss_by_epoch.keys())))

df = pd.DataFrame({
    "epoch": epochs,
    "train_loss": [train_loss_by_epoch.get(e, None) for e in epochs],
    "val_loss": [val_loss_by_epoch.get(e, None) for e in epochs],
})

tsv_path = f"{OUTPUT_DIR}/train_val_loss.tsv"
df.to_csv(tsv_path, sep="\t", index=False)

print("[OK] Saved TSV:", tsv_path)
display(df)

# push TSV lên hub (nằm trong output_dir sẽ được push)
trainer.push_to_hub(commit_message="Add train_val_loss.tsv")
print("[OK] Pushed TSV to hub.")

[OK] Saved TSV: mbart50_envi/train_val_loss.tsv


,epoch,train_loss,val_loss
0,1.0,1.9404,1.723647
1,2.0,1.5553,1.619265
2,3.0,1.3820,1.573678
3,4.0,1.2750,1.555019
4,5.0,1.1969,1.545686
5,6.0,1.1411,1.545778
6,7.0,1.1068,1.545686


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


[OK] Pushed TSV to hub.


# 📈 CELL 17 — Tính BLEU trên Public Test và Push lên Hub

## Mục đích
Đánh giá model đã train trên toàn bộ tập public test (2,943 câu) và lưu kết quả lên Hugging Face.

## Hàm `bleu_on_public_test_and_push()`

### Quy trình đánh giá:

1. **Load dữ liệu test**
   ```python
   srcs = ds["test"]["src_text"]  # 2,943 câu tiếng Anh
   refs = ds["test"]["tgt_text"]  # 2,943 reference tiếng Việt
   ```

2. **Dịch theo batch**
   - Batch size: 16 (trên GPU)
   - Dùng beam search với num_beams=5
   - Progress bar với tqdm

3. **Tính BLEU**
   ```python
   bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
   ```
   - Corpus BLEU: đánh giá trên toàn bộ tập, không phải trung bình
   - Chính xác hơn sentence-level BLEU

4. **Lưu kết quả**
   - `public_test_metrics.json`: BLEU, size, batch_size, time
   - `public_test_bleu.txt`: Chỉ điểm BLEU
   - Push files lên Hugging Face repo

## Kết quả Thí nghiệm 2:

```
[PUBLIC_TEST] sacreBLEU (2943 samples): 38.09
```

### So sánh với Thí nghiệm 1:

| Metric | Thí nghiệm 1 | Thí nghiệm 2 | Chênh lệch |
|--------|--------------|--------------|------------|
| Data size | 20K | 10K | -50% |
| Epochs | 3 | 7 (best at 5) | +133% |
| Learning rate | 3e-5 | 1e-5 | -67% |
| BLEU score | **41.51** | **38.09** | **-3.42** |

### Phân tích:
- **Tác động dữ liệu**: Giảm 50% data → giảm 8.2% BLEU
- **Trade-off**: Nhiều data quan trọng hơn nhiều epochs
- **Hiệu quả**: 10K mẫu vẫn đạt 38.09 BLEU (vượt baseline 26.20)

## Files output:
- `/kaggle/working/mbart50_envi/public_test_metrics.json`
- `/kaggle/working/mbart50_envi/public_test_bleu.txt`
- Pushed to: `https://huggingface.co/ngothuyet/mbart50-envi-e7`

In [16]:
import torch
from typing import List

MAX_SRC_LEN = 256

@torch.inference_mode()
def translate_en2vi(
    texts: List[str],
    num_beams: int = 5,
    length_penalty: float = 1.0,
    no_repeat_ngram_size: int = 3,
    max_new_tokens: int = 128,
):
    model.eval()
    tokenizer.src_lang = SRC_LANG
    tokenizer.tgt_lang = TGT_LANG

    enc = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_SRC_LEN
    ).to(model.device)

    generated = model.generate(
        **enc,
        forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG],
        num_beams=num_beams,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        max_new_tokens=max_new_tokens,
        early_stopping=True,
    )

    return tokenizer.batch_decode(generated, skip_special_tokens=True)

# Demo nhanh
demo = [
    "It's necessary to provide health insurance communication and education for people who live in remote areas and participate interupted health insurance.",
    "I am learning natural language processing."
]
print("[INPUT ]", demo)
print("[OUTPUT]", translate_en2vi(demo, num_beams=5, length_penalty=1.0, no_repeat_ngram_size=3))

[INPUT ] ["It's necessary to provide health insurance communication and education for people who live in remote areas and participate interupted health insurance.", 'I am learning natural language processing.']
[OUTPUT] ['Cần cung cấp dịch vụ bảo hiểm y tế, truyền thông và giáo dục cho những người sống ở những vùng xa xôi và tham gia bảo hiểm Y tế liên kết.', 'Tôi đang học xử lý ngôn ngữ tự nhiên.']


In [17]:
from datasets import Dataset

# đọc song song public_test.en/vi -> list[(en, vi)]
test_pairs_raw = load_parallel(TEST_EN, TEST_VI, "test_raw")

# clean/filter/dedup giống pipeline train
test_pairs = clean_filter_dedup(test_pairs_raw, "test")

print(f"[INFO] public_test after clean: {len(test_pairs):,} pairs")
print("[SAMPLE TEST]", test_pairs[0])

# tạo ds_test (giữ nguyên ds train/val của anh nếu đã có)
test_ds = Dataset.from_dict({
    "src_text": [s for s, _ in test_pairs],
    "tgt_text": [t for _, t in test_pairs],
})

# gắn vào DatasetDict nếu đã có ds, còn không thì tạo ds mới
if "ds" in globals():
    ds["test"] = test_ds
else:
    from datasets import DatasetDict
    ds = DatasetDict({"test": test_ds})

print("[OK] ds['test'] size:", len(ds["test"]))

[INFO] test_raw lines -> 3,000 pairs.
[INFO] test: after clean+filter -> 2,946 pairs
[INFO] test: after dedup       -> 2,943 pairs
[INFO] public_test after clean: 2,943 pairs
[SAMPLE TEST] ('Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao', 'Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017')
[OK] ds['test'] size: 2943


In [18]:
import os, json, time
import torch
import sacrebleu
from tqdm.auto import tqdm

assert "ds" in globals() and "test" in ds, "Missing ds['test'] - run TEST-LOAD cell first."
assert "translate_en2vi" in globals(), "Missing translate_en2vi() - run inference cell first."
assert "OUTPUT_DIR" in globals(), "Missing OUTPUT_DIR (e.g., 'mbart50_envi')."
assert "trainer" in globals(), "Missing trainer - run training cell first (to push easily)."

def bleu_on_public_test_and_push(batch_size=16):
    srcs = ds["test"]["src_text"]
    refs = ds["test"]["tgt_text"]

    bs = batch_size if torch.cuda.is_available() else max(2, batch_size // 4)

    hyps = []
    t0 = time.time()

    for i in tqdm(range(0, len(srcs), bs), desc="BLEU on public_test", total=(len(srcs) + bs - 1)//bs):
        batch = srcs[i:i+bs]
        hyps.extend(
            translate_en2vi(
                batch,
                num_beams=5,
                length_penalty=1.0,
                no_repeat_ngram_size=3,
                max_new_tokens=128
            )
        )

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    elapsed = time.time() - t0

    metrics = {
        "public_test_sacrebleu": float(bleu),
        "public_test_size": int(len(srcs)),
        "batch_size": int(bs),
        "elapsed_sec": float(elapsed),
    }

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    json_path = os.path.join(OUTPUT_DIR, "public_test_metrics.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)

    txt_path = os.path.join(OUTPUT_DIR, "public_test_bleu.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"public_test_sacrebleu\t{bleu:.4f}\n")

    print(f"\n[PUBLIC_TEST] sacreBLEU ({len(srcs)} samples): {bleu:.2f}")
    print("[OK] Saved:", json_path)
    print("[OK] Saved:", txt_path)

    # push lên repo model
    trainer.push_to_hub(commit_message=f"Add public_test sacreBLEU={bleu:.2f}")
    print("[OK] Pushed public_test BLEU files to Hugging Face.")

    return bleu, metrics

_ = bleu_on_public_test_and_push(batch_size=16)

BLEU on public_test:   0%|          | 0/184 [00:00<?, ?it/s]


[PUBLIC_TEST] sacreBLEU (2943 samples): 38.09
[OK] Saved: mbart50_envi/public_test_metrics.json
[OK] Saved: mbart50_envi/public_test_bleu.txt


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[OK] Pushed public_test BLEU files to Hugging Face.
